In [1]:
import $ivy.`com.stripe::rainier-core:0.3.2`
import $ivy.`com.stripe::rainier-notebook:0.3.3`

import $ivy.$                               

import $ivy.$                                   

In [2]:
import com.stripe.rainier.compute._
import com.stripe.rainier.core._
import com.stripe.rainier.sampler._
import com.stripe.rainier.notebook._

import com.stripe.rainier.compute._

import com.stripe.rainier.core._

import com.stripe.rainier.sampler._

import com.stripe.rainier.notebook._

In [3]:
PPrint.register(repl)

In [4]:
import $ivy.`com.github.tototoshi::scala-csv:1.3.6`
import com.github.tototoshi.csv._
import java.io.File

import $ivy.$                                      

import com.github.tototoshi.csv._

import java.io.File

In [5]:
def time[R](block: => R): R = {
    val t0 = System.nanoTime()
    val result = block    // call-by-name
    val t1 = System.nanoTime()
    val timeMilis = ((t1 - t0) / 1e6).toInt
    println(s"Elapsed time: $timeMilis ms")
    result
}

defined function time

In [6]:
val countriesOnsetData = {
    val csvReader = CSVReader.open(new File("data/onset.csv"))
    val xs = csvReader.allWithHeaders()
    csvReader.close()
    xs
}

countriesOnsetData: List[Map[String, String]] = List(
  Map("Country" -> "Austria", "Date" -> "2019-10-27", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-10-28", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-10-29", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-10-30", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-10-31", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-01", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-02", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-03", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-04", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-05", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-06", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-07", "Onset" -> "0.0"),
  Map("Country" -> "Austria", "Date" -> "2019-11-08", "Onset" -> "0.0"

In [8]:
val pdelayData = {
    val csvReader = CSVReader.open(new File("data/pdelay.csv"))
    val xs = csvReader.allWithHeaders()
    csvReader.close()
    xs
}

pdelayData: List[Map[String, String]] = List(
  Map("index" -> "0", "pdelay" -> "0.006547041707080504"),
  Map("index" -> "1", "pdelay" -> "0.04194956353055286"),
  Map("index" -> "2", "pdelay" -> "0.05771096023278371"),
  Map("index" -> "3", "pdelay" -> "0.06983511154219205"),
  Map("index" -> "4", "pdelay" -> "0.07153249272550921"),
  Map("index" -> "5", "pdelay" -> "0.08583899127061105"),
  Map("index" -> "6", "pdelay" -> "0.07953443258971872"),
  Map("index" -> "7", "pdelay" -> "0.07516973811833172"),
  Map("index" -> "8", "pdelay" -> "0.07710960232783705"),
  Map("index" -> "9", "pdelay" -> "0.07904946653734239"),
  Map("index" -> "10", "pdelay" -> "0.06304558680892337"),
  Map("index" -> "11", "pdelay" -> "0.0511639185257032"),
  Map("index" -> "12", "pdelay" -> "0.04485935984481086"),
  Map("index" -> "13", "pdelay" -> "0.038554801163918524"),
  Map("index" -> "14", "pdelay" -> "0.028855480116391854"),
  Map("index" -> "15", "pdelay" -> "0.026430649854510185"),
  Map("index" -> 

In [15]:
val pdelay = pdelayData.collect { case m => m("pdelay").toDouble }

pdelay: List[Double] = List(
  0.006547041707080504,
  0.04194956353055286,
  0.05771096023278371,
  0.06983511154219205,
  0.07153249272550921,
  0.08583899127061105,
  0.07953443258971872,
  0.07516973811833172,
  0.07710960232783705,
  0.07904946653734239,
  0.06304558680892337,
  0.0511639185257032,
  0.04485935984481086,
  0.038554801163918524,
  0.028855480116391854,
  0.026430649854510185,
  0.01915615906886518,
  0.02061105722599418,
  0.011881668283220174,
  0.009456838021338506,
  0.009699321047526674,
  0.006547041707080504,
  0.004122211445198836,
  0.0036372453928225023,
  0.002909796314258002,
  0.0036372453928225023,
  0.0019398642095053346,
  4.8496605237633366E-4,
  4.8496605237633366E-4,
  0.0,
  9.699321047526673E-4,
  2.4248302618816683E-4,
  4.8496605237633366E-4,
  7.274490785645005E-4,
  0.001454898157129001,
  2.4248302618816683E-4,
  4.8496605237633366E-4,
  7.274490785645005E-4,
...

Preparation of the other model

In [9]:
import scala.collection.mutable.ListBuffer

def prepareDataForModelOnset(xs: List[Long], pd: List[Double], window: Int): List[(Long,Double)] = {
    val cpdBuffer = new ListBuffer[Double]()
    cpdBuffer ++= pd.scanLeft(0.0)(_+_).drop(1)
    for(i <- 1 to xs.size - pd.size) {
        cpdBuffer += 1.0
    }
    val temp = xs.zip(cpdBuffer.toList.reverse)
    temp.drop(temp.size - window)
}

import scala.collection.mutable.ListBuffer


defined function prepareDataForModelOnset

In [10]:
val samplerConfig = new DefaultConfig {
    override def massMatrixTuner() = new IdentityMassMatrixTuner
    override def sampler() = new HMCSampler(100)
}

samplerConfig: DefaultConfig{def massMatrixTuner(): com.stripe.rainier.sampler.IdentityMassMatrixTuner;def sampler(): com.stripe.rainier.sampler.HMCSampler} = ammonite.$sess.cmd9$Helper$$anon$1@16cf331e

Define a function that we'll use to retry the call to Rainier's `predict` that fails occassionally. Fortunately, the failure can be easily detected and 1-2 retries solves the issue.

In [11]:
def retry[T](maxAttempts: Int, isSuccessful: T => Boolean)(f: => T): T = {
    var i = 0
    
    do {
        val res = f
        i = i + 1
        if (isSuccessful(res))
            return res
        println(s"WARN: Attempt[$i] failed. Retrying...")
    } while (i < maxAttempts)
    sys.error("Exceeded maximum number of retries")
}

defined function retry

In [12]:
def predictRtOnset(onsetNumbers: List[Long], pdelay: List[Double], window: Int): (Vec[Real], List[Seq[Double]]) = {
    
    val data = prepareDataForModelOnset(onsetNumbers, pdelay, window)
    
    val (model, rts) = {
        val (onset, cum_pdelay) = data.unzip
        val serialInterval = Gamma(6,1/1.5).latent
        val sigma = Normal(0,0.03).latent.abs
        val Theta0 = Normal(0.1,0.1).latent
        val diffs = Laplace(0,sigma).latentVec(data.size) // Maybe normal would be better
        val ThetaT = Vec.from(diffs.toList.scanLeft(Theta0)(_ + _))
        /*
        If we remove multiplication by a rv serialInterval the crash disappears
        */
        val Rts = ThetaT.map { m =>  
            m*serialInterval + 1
        }
        val inferredYesterday = data.take(data.size-1).map { case (o_t, cp_t) => 
            o_t/cp_t 
        }.zipWithIndex
        val expectedToday = Vec.from(inferredYesterday.zip(cum_pdelay.drop(1))).map { case ((infYest, i), cpd) =>
            val lambda = infYest * cpd * (ThetaT(i).exp)
            Poisson(lambda)
        }
        (Model.observe(onset.drop(1), expectedToday), Rts)
    }
    
    def detectFailedRun(trace: Trace): Boolean =
        // from: https://twitter.com/avibryant/status/1253049509128499201
        trace.diagnostics.map(_.rHat).max < 1.1
    
    val trace = retry(maxAttempts = 3, isSuccessful = detectFailedRun) {
        println("Retrying..")
        model.sample(samplerConfig, 2)
    }
    println("Sampled..")
    val tracePredict = trace.predict(rts)
    println("tracePredict prepared..")
    return rts -> tracePredict
}

defined function predictRtOnset

In [13]:
case class RtsDistributionStats(means: Seq[(Int, Double)], hdpis: Seq[(Int, (Double, Double))])
def distributionStats(rts: Vec[Real], rtForecasts: List[Seq[Double]]): RtsDistributionStats = {
    val means = 0.until(rts.size).map{i => (i,mean(rtForecasts.map(_(i))))}
    val hdpis = 0.until(rts.size).map{i => (i, hdpi(rtForecasts.map(_(i))))}
    RtsDistributionStats(means, hdpis)
}

defined class RtsDistributionStats
defined function distributionStats

In [16]:
{{
    val countryName = "Poland"
    val onsetNumbers = countriesOnsetData.collect {
        case m if m("Country") == countryName => m("Onset").toDouble.round
    }
    val (rts,estimates) = time { 
        val res = predictRtOnset(onsetNumbers, pdelay, 30)
        print("Returning..")
        res
    }
    print("Here[1]..")
    val RtsDistributionStats(means, hdpis) = distributionStats(rts, estimates)
    print("Here[2]..")
    show("t", "Rt", line(means), shade(hdpis))
}}

Retrying..


Chain 1 Complete : Iteration: 1000 (7ms) Acceptance rate: 0.80 E-BFMI: 0.64 Step size: 0.03 Total gradient evaluations: 2e+05 (6us) Total time: 42s

Chain 2 Complete : Iteration: 1000 (10ms) Acceptance rate: 0.85 E-BFMI: 0.71 Step size: 0.03 Total gradient evaluations: 2e+05 (11us) Total time: 31s

Sampled..


: 